In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/water-pollution/Submission.csv
/kaggle/input/water-pollution/Target.csv
/kaggle/input/water-pollution/Train.csv
/kaggle/input/water-pollution/Test.csv
/kaggle/working/__notebook__.ipynb


In [2]:

! touch /root/.kaggle/kaggle.json
! echo '{"username":"hydramst","key":"cf8c1eb62bc50744b3798156fb56acfc"}' > /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json
! cat /root/.kaggle/kaggle.json

touch: cannot touch '/root/.kaggle/kaggle.json': No such file or directory
/bin/bash: /root/.kaggle/kaggle.json: No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
cat: /root/.kaggle/kaggle.json: No such file or directory


In [3]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/Train.csv', delimiter = ',')
DF_train

,Unnamed: 0,code,year,period,id,Country,tourists,venue,rate,food,glass,metal,other,paper,plastic,leather,green_waste,waste_recycling
0,6,CAS_7723-14-0,2009,2009-01--2009-12,ES014MSPFES-014-NR-038-000-02-02,Spain,50941692.0,4.0,87.158924,49.0,8.0,3.0,14.0,15.0,9.0,0.0,0.0,16.84
1,10,CAS_7723-14-0,2009,2009-01--2009-12,FRGR0220,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2,17,CAS_14265-44-2,2009,2009-01--2009-12,FRGR1496,France,71176346.0,4.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
3,24,CAS_7723-14-0,2009,2009-01--2009-12,FRDR174,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
4,34,CAS_7723-14-0,2009,2009-01--2009-12,FRGR1022,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,19877,CAS_14265-44-2,2009,2009-01--2009-12,UKSC006555,United Kingdom,25867961.0,0.0,0.000000,16.7,2.2,3.5,28.2,18.9,20.2,0.0,2.7,27.25
2314,19890,CAS_7723-14-0,2009,2009-01--2009-12,FRGR0380,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2315,19895,CAS_14265-44-2,2009,2009-01--2009-12,ES080MSPF18-28,Spain,50941692.0,0.0,87.158924,49.0,8.0,3.0,14.0,15.0,9.0,0.0,0.0,16.84
2316,19899,CAS_7723-14-0,2009,2009-01--2009-12,FRHR251A,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26


In [4]:
DF_test = pd.read_csv('/kaggle/input/water-pollution/Test.csv', delimiter = ',')
DF_test

,Unnamed: 0,code,year,period,id,Country,tourists,venue,rate,food,glass,metal,other,paper,plastic,leather,green_waste,waste_recycling
0,402,CAS_7723-14-0,2009,2009-01--2009-12,FRHR326,France,71176346.0,0.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
1,748,CAS_7723-14-0,2008,2008-01--2008-12,FRHR80,France,71176346.0,0.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2,763,CAS_7723-14-0,2008,2008-01--2008-12,FRHR79,France,71176346.0,1.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
3,942,CAS_14265-44-2,2009,2009-01--2009-12,DERW_DENW2742_0_16,Germany,22635423.0,4.0,0.0,30.0,10.0,1.4,17.7,24.0,13.0,4.0,0.0,47.83
4,1258,CAS_14265-44-2,2008,2008-01--2008-12,FRDR509C,France,71176346.0,0.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,19447,CAS_7723-14-0,2010,2010-01--2010-12,UKSC010040,United Kingdom,25867961.0,0.0,0.0,16.7,2.2,3.5,28.2,18.9,20.2,0.0,2.7,27.25
118,19458,CAS_7723-14-0,2010,2010-01--2010-12,FRHR216C,France,71176346.0,0.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
119,19464,CAS_14265-44-2,2010,2010-01--2010-12,FRFR201,France,71176346.0,0.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
120,19699,CAS_7723-14-0,2010,2010-01--2010-12,FRGR0366B,France,71176346.0,0.0,0.0,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26


In [5]:
Target = pd.read_csv('/kaggle/input/water-pollution/Target.csv', delimiter = ',')
Target

,Unnamed: 0,polution
0,6,0.013212
1,10,0.057482
2,17,0.037835
3,24,0.055119
4,34,0.037617
...,...,...
2313,19877,0.056002
2314,19890,0.105636
2315,19895,0.002696
2316,19899,0.087706


In [6]:
Submission = pd.read_csv('/kaggle/input/water-pollution/Submission.csv', delimiter = ',')
Submission

,idx,polution
0,402,0.104649
1,748,0.104649
2,763,0.104649
3,942,0.104649
4,1258,0.104649
...,...,...
117,19447,0.104649
118,19458,0.104649
119,19464,0.104649
120,19699,0.104649


In [7]:
DF_total = pd.concat([DF_train, DF_test])
DF_total = DF_total.drop(columns = 'Unnamed: 0', axis =1 )
DF_total

,code,year,period,id,Country,tourists,venue,rate,food,glass,metal,other,paper,plastic,leather,green_waste,waste_recycling
0,CAS_7723-14-0,2009,2009-01--2009-12,ES014MSPFES-014-NR-038-000-02-02,Spain,50941692.0,4.0,87.158924,49.0,8.0,3.0,14.0,15.0,9.0,0.0,0.0,16.84
1,CAS_7723-14-0,2009,2009-01--2009-12,FRGR0220,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2,CAS_14265-44-2,2009,2009-01--2009-12,FRGR1496,France,71176346.0,4.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
3,CAS_7723-14-0,2009,2009-01--2009-12,FRDR174,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
4,CAS_7723-14-0,2009,2009-01--2009-12,FRGR1022,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,CAS_7723-14-0,2010,2010-01--2010-12,UKSC010040,United Kingdom,25867961.0,0.0,0.000000,16.7,2.2,3.5,28.2,18.9,20.2,0.0,2.7,27.25
118,CAS_7723-14-0,2010,2010-01--2010-12,FRHR216C,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
119,CAS_14265-44-2,2010,2010-01--2010-12,FRFR201,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
120,CAS_7723-14-0,2010,2010-01--2010-12,FRGR0366B,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26


In [8]:
cat_columns = []
num_columns = []

for column_name in DF_total.columns:
    if (DF_total[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['code', 'period', 'id', 'Country'] 
 len =  4
numerical columns:	  ['year', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
 len =  13


In [9]:
train = DF_total.iloc[0:DF_train.shape[0],:]
test = DF_total.iloc[DF_train.shape[0]:,:]

In [10]:
DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0], 

(2318, 2318, 122, 122)

In [11]:
# Only numeric

X_train = train[num_columns].values
X_test = test[num_columns].values

y_train = Target['polution'].values.ravel()


In [12]:
# Train / validate separation 

from sklearn.model_selection import train_test_split

In [13]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train,y_train,test_size = 0.2, random_state = 42)

In [14]:
some_prediction = np.random.rand(122,1)

In [15]:
Submission['polution'] = some_prediction
Submission

,idx,polution
0,402,0.899528
1,748,0.937337
2,763,0.673691
3,942,0.561467
4,1258,0.283776
...,...,...
117,19447,0.774913
118,19458,0.864262
119,19464,0.094443
120,19699,0.895481


In [16]:
mkdir /kaggle/working/water-pollution

In [17]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [18]:
!kaggle competitions submit -c sf-matml-2022-regression -f '/kaggle/working/water-pollution/Submission.csv' -m "Test"

Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.7/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
